In [694]:
 np.set_printoptions(formatter={'float': lambda x: "{0:0.10f}".format(x)}) #Formato para ver 10 decimales


# Modelos vs Realidad
Ustedes como ingenieros aprenden muchos modelos de la realidad. Pero algunas veces no quedan claros los límites de los modelos. Y cuando aplican esos modelos fuera de los límites ¡oh! todo falla.  Es en estos casos cuando nuestros modelos deben mejorar.

# Problema con el ejercicio anterior

En el problema anterior se solicitaba la forma escalón reducida y alguna solución particular básica entre otras cosas, de una matriz extendida como la siguiente. 

In [695]:
import numpy as np
import copy

matriz_pregunta=np.matrix([ 
    [11, 2, 13,  3, 0, 1],
    [4,  15, 19, 6, 1, 2],
    [7,  8, 15, 19, 1, 3]
])

Al implementar el algoritmo de Gauss se obtiene la siguiente matriz.

In [696]:
A=np.matrix([[1,        0, 44073040/44073040,        0,  -892859/44073040,  2089670/44073040],
        [       0, 1, -4006640/-4006640,        0,  -231418/-4006640,  -310860/-4006640],
        [       0,        0,        0,    1,      913/25520,     2750/25520]])

A

matrix([[1.0000000000, 0.0000000000, 1.0000000000, 0.0000000000,
         -0.0202586207, 0.0474137931],
        [0.0000000000, 1.0000000000, 1.0000000000, 0.0000000000,
         0.0577586207, 0.0775862069],
        [0.0000000000, 0.0000000000, 0.0000000000, 1.0000000000,
         0.0357758621, 0.1077586207]])

El exceso de dígitos decimales impide visualizar la matriz de manera ordenada. Esto se puede solucionar disminuyendo los decimales que se presenta con la siguiente instrucción.  

In [697]:
np.set_printoptions(formatter={'float': lambda x: "{0:0.3f}".format(x)}) #Formato para ver 3 decimales

A

matrix([[1.000, 0.000, 1.000, 0.000, -0.020, 0.047],
        [0.000, 1.000, 1.000, 0.000, 0.058, 0.078],
        [0.000, 0.000, 0.000, 1.000, 0.036, 0.108]])

Ahora sí es más sencillo ver la matriz.

La  solución particular trivial, es decir cuando los parámetros son $t_0=0$ y $t_1=0$, es la siguiente.

In [698]:
(x0,x1,x2,x3,x4)=(0.047, 0.078, 0, 0.108, 0)

(x0,x1,x2,x3,x4)

(0.047, 0.078, 0, 0.108, 0)

Ahora verificamos si la solución particular es correcta 


In [699]:
11*x0 + 2*x1 + 13*x2 + 3*x3 + 0*x4 == 1

False

Ups, algo fallo! veamos cuanto dá la suma

In [700]:
11*x0 + 2*x1 + 13*x2 + 3*x3 + 0*x4

0.9970000000000001

Da casi 1, el error es debido a la aproximación que se hizo para ver mejor la matriz. Para obtener la igualdad debemos colocar todos los decimales. ¿Pero si el número tiene infinitos decimales, cómo los podemos colocar todos?

# Método numérico

Ya que no se pueden colocar todos los decimales de algunos resultados entonces vale la pena explorar otros métodos que nos permiten aproximar la respuesta. En [Nakos, sec 2.3] se presentan dos métodos que permiten aproximar la respuesta tanto como queramos. Estos métodos no son para cualquier sistema de ecuaciones sino solamente para aquellos con solución única cuya matriz de coeficientes sea cuadrada. Pero para matrices grandes puede llegar a ser más rápido.

A continuación vamos a conocer un poco mejor como se modelan los números en Python y cómo poder representar la solución exacta de cualquier sistema de ecuaciones.

# Modelo de los números enteros y de punto flotante

Ni el conjunto de los números naturales, ni el conjunto de los números reales se puede implementar en un computador. Ya que estos conjuntos tienen infinitos elementos y la memoria del computador es finita. 

En Python los **números enteros** se modelan con el tipo de dato `int`. Estos no están limitados por Python sino por la memoria del computador. En principio podemos hacer operaciones con números enteros con tranquilidad.  

In [701]:
(10**100)+1

10000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000001

En Python el conjunto de los **números racionales** se puede modelar con la librería `fraction`. Por ejemplo, la fracción $\frac{9}{6}$ se puede escribir `Fraction(9,6)` y será reducida.  

En vez de `Fraction` se puede usar un alias como `frac`

In [702]:
from fractions import Fraction as frac
n=frac(9,6)
n

Fraction(3, 2)

Con el fin de mejorar la presentación, se puede usar la función `pprint()`. 

In [703]:
pprint(n)

3/2


In [704]:
pprint(frac(1000,998))

500
───
499


In [705]:
pprint(frac(15**50,-6**50))

-88817841970012523233890533447265625 
─────────────────────────────────────
           1125899906842624          


El modelo de los **números reales** es con el tipo de dato `float`, los cuales se escriben con el punto decimal, como $1.0$. Este tipo de dato es muchísimo más limitado debido a la forma en que Python los almacena. Ya que el usuario escribe y lee en base 10, es decir algo así $a(10^b)$. Es to se denota en Python con la letra `e`

In [706]:
1.25*(10**128)

1.25e+128

Pero Python almacena y opera en base 2, es decir así $a'(2^{b'})$. Algunas operaciones que son evidentes en base 10 pueden tener infinitos decimales en base 2 y al hacer las aproximaciones de estos decimales terminan generando un error en base 10. Como en los siguientes ejemplos. 

In [707]:
5.7*(10**100)

5.700000000000001e+100

In [708]:
0.1 + 0.1 + 0.1 - 0.3

5.551115123125783e-17

Otro problema es que en `float`, la representación  $a'(2^{b'})$ tiene límites para $a'$ y para $b'$. Lo cual genera el siguiente error.

In [709]:
(10**100)+1.0

1e+100

Por las anteriores razones trataremos de evitar el tipo de dato `float`.

La librería `decimal` es un nuevo modelo de los números reales que pretende solucionar algunos de los problemas de  `float` pero no la usaremos en este curso.

Hay muchísimas más cosas que se pueden decir de todos estos tipos de datos, pero queda fuera del alcance de este curso. Esto es tema de los cursos de programación y de fundamentos de circuitos digitales.

# Eliminación de Gauss - Jordan.


La eliminación de Gauss - Jordan es similar a la eliminación de Gauss pero no sólo elimina los coeficientes abajo del pivote sino que también elimina los coeficiente arriba del pivote en el mismo paso. Para evitar los fraccionarios usaremos la operación entre renglones que está formada por dos operaciones elementales.

$k_i R_i + k_j R_j \rightarrow R_j$

Para ilustrar el procedimiento se realizará el siguiente ejemplo.

# Ejemplo.

Encuentre la solución general, la solución particular trivial y las soluciones particulares básicas que haya, de la matriz extendida `matriz_pregunta`.


In [710]:
import numpy as np
import copy

matriz_pregunta=np.matrix([ 
    [11, 2, 13,  3, 0, 1],
    [4,  15, 19, 6, 1, 2],
    [7,  8, 15, 19, 1, 3]
])




## Solución
Primero hacemos una copia de `matriz_pregunta` para no afectarla. Esta copia no se puede hacer simplemente con el igual, `A=matriz_pregunta`, ya que ambas variables se referirían a la misma matriz. Por este motivo usamos la función `deepcopy`, que hace una copia de todos los objetos mutables que pueda tener el arreglo.

In [711]:
A=copy.deepcopy(matriz_pregunta)

Empezamos con la columna 0.

In [712]:
# 4R0 - 11R1 -> R1
A[1,:] = 4*A[0,:] - 11*A[1,:]
A

matrix([[  11,    2,   13,    3,    0,    1],
        [   0, -157, -157,  -54,  -11,  -18],
        [   7,    8,   15,   19,    1,    3]])

In [713]:
# 7R0 - 11R2 -> R2
A[2,:] = 7*A[0,:] - 11*A[2,:]
A

matrix([[  11,    2,   13,    3,    0,    1],
        [   0, -157, -157,  -54,  -11,  -18],
        [   0,  -74,  -74, -188,  -11,  -26]])

Ahora continuamos con la columna 1.

In [714]:
# 74R1 - 157R2 -> R2
A[2,:] = 74*A[1,:] - 157*A[2,:]
A

matrix([[   11,     2,    13,     3,     0,     1],
        [    0,  -157,  -157,   -54,   -11,   -18],
        [    0,     0,     0, 25520,   913,  2750]])

In [715]:
# 2R1 + 157R0 -> R0
A[0,:] = 2*A[1,:] + 157*A[0,:]
A

matrix([[ 1727,     0,  1727,   363,   -22,   121],
        [    0,  -157,  -157,   -54,   -11,   -18],
        [    0,     0,     0, 25520,   913,  2750]])

La columna 2 no tiene pivote, entonces pasamos a la columan 3.

In [716]:
A[1,:] = 54*A[2,:] + 25520*A[1,:]
A

matrix([[    1727,        0,     1727,      363,      -22,      121],
        [       0, -4006640, -4006640,        0,  -231418,  -310860],
        [       0,        0,        0,    25520,      913,     2750]])

In [717]:
A[0,:] = -363*A[2,:] + 25520*A[0,:]
A

matrix([[44073040,        0, 44073040,        0,  -892859,  2089670],
        [       0, -4006640, -4006640,        0,  -231418,  -310860],
        [       0,        0,        0,    25520,      913,     2750]])

El sistema de ecuaciones escalón reducido sin simplificar es:

$\matrix{
44073040 x_0 &    + & 44073040 x_2 & - & 892859 x_4 & = &  2089670 \\
-4006640 x_1  &  - &  4006640 x_2  & - & 231418 x_4 & = &  -310860\\
               &   & 25520 x_3   &  +  & 913 x_4 & = &  2750}$
               
Este sistema como no tiene pivotes en los términos constantes entonces es <u>consistente</u>.

Como tiene variables libres entonces tiene <u>infinitas soluciones</u>. 

Como $x_2, x_4$ son una variables libres, entonces les asignamos un parámetro a cada una, $x_2=t_0, x_4=t_1$.
Al despejar las variables delanteras se obtiene:

$\matrix{
    x0 & = & \frac{2089670}{44073040)}& - & t_0 * \frac{44073040}{44073040} & - & t_1 * \frac{-892859}{44073040} \\
    x1 & = & \frac{-310860}{-4006640} & - & t_0 * \frac{-4006640}{-4006640} & - & t_1 * \frac{-231418}{-4006640} \\
    x2 & = & t_0 \\
    x3 & = & \frac{2750}{2552}& - & t_1 * \frac{913}{25520} \\
    x4 & = & t_1}$

La solución general se va a escribir como una función en términos de los parámetros.

Para evitar los decimales y sus problemas al verificar la respuesta,
usaremos los fraccionarios de Python.

In [718]:
from fractions import Fraction as frac # Librería para el uso de fraccionarios
                                       # frac(numerador,denominador)

def solucion_general(t0,t1):
    x0 = frac(2089670,44073040) - t0 * frac(44073040,44073040)  - t1 * frac(-892859,44073040)
    x1 = frac(-310860,-4006640) - t0 * frac(-4006640 ,-4006640) -  t1 * frac(-231418,-4006640) 
    x2 = t0
    x3 =  frac(2750,25520)  -  t1 * frac(913,25520)
    x4 = t1
    return (x0,x1,x2,x3,x4)

Ahora verificamos la solución particular trivial, con $t_0=0$ y $t_1=0$.

In [719]:
(x0,x1,x2,x3,x4)=solucion_general(0,0)
pprint((x0,x1,x2,x3,x4))  # pprint mejora la presentación de los fraccionarios 

⎛ 11             25   ⎞
⎜───, 9/116, 0, ───, 0⎟
⎝232            232   ⎠


In [720]:
(x0,x1,x2,x3,x4)

(Fraction(11, 232), Fraction(9, 116), 0, Fraction(25, 232), 0)

In [722]:
matriz_pregunta # Se imprime la matriz original para ver los coeficientes

matrix([[11,  2, 13,  3,  0,  1],
        [ 4, 15, 19,  6,  1,  2],
        [ 7,  8, 15, 19,  1,  3]])

In [723]:
11*x0 + 2*x1 + 13*x2 + 3*x3 + 0*x4 == 1

True

In [724]:
4*x0 + 15*x1 + 19*x2 + 6*x3 + 1*x4 == 2

True

In [725]:
7*x0 + 8*x1 + 15*x2 + 19*x3 + 1*x4 == 3

True

Seguimos con la verificación de la primera solución particular básica, con $t_0=1$ y $t_1=0$.

In [726]:
(x0,x1,x2,x3,x4)=solucion_general(1,0)
pprint((x0,x1,x2,x3,x4))  # pprint mejora la presentación de los fraccionarios 

⎛-221   -107       25   ⎞
⎜─────, ─────, 1, ───, 0⎟
⎝ 232    116      232   ⎠


In [727]:
(x0,x1,x2,x3,x4)

(Fraction(-221, 232), Fraction(-107, 116), 1, Fraction(25, 232), 0)

In [729]:
11*x0 + 2*x1 + 13*x2 + 3*x3 + 0*x4 == 1

True

In [730]:
4*x0 + 15*x1 + 19*x2 + 6*x3 + 1*x4 == 2

True

In [731]:
7*x0 + 8*x1 + 15*x2 + 19*x3 + 1*x4 == 3

True

Finalmente se verifica la segunda solución particular básica, con $t_0=0$ y $t_1=1$.

In [732]:
(x0,x1,x2,x3,x4)=solucion_general(0,1)
pprint((x0,x1,x2,x3,x4))  # pprint mejora la presentación de los fraccionarios 

⎛157    23      167    ⎞
⎜────, ────, 0, ────, 1⎟
⎝2320  1160     2320   ⎠


In [733]:
11*x0 + 2*x1 + 13*x2 + 3*x3 + 0*x4 == 1

True

In [734]:
4*x0 + 15*x1 + 19*x2 + 6*x3 + 1*x4 == 2

True

In [735]:
7*x0 + 8*x1 + 15*x2 + 19*x3 + 1*x4 == 3

True

Finalmente se escribe la tupla de las tuplas de las soluciones particulares.

In [736]:
soluciones_particulares=(
    solucion_general(0,0),
    solucion_general(1,0),
    solucion_general(0,1)
)

soluciones_particulares

((Fraction(11, 232), Fraction(9, 116), 0, Fraction(25, 232), 0),
 (Fraction(-221, 232), Fraction(-107, 116), 1, Fraction(25, 232), 0),
 (Fraction(157, 2320), Fraction(23, 1160), 0, Fraction(167, 2320), 1))

# Ejercicio

In [737]:
nombre= '  escriba aquí su nombre   '
código=0    # reemplazar el cero por su código

## Ejercicio

Utilizando el algoritmo de Gauss, encuentre:
* Una rutina llamada `solucion_general(parametros)` con la solución general del siguiente sistema de ecuaciones. 
* En la variable `soluciones_particulares` asigne una tupla con la tupla de la solución particular trivial y las tuplas de las soluciones particulares básicas, si existen.

$\matrix{
-2x_1 + ax_2 - bx_3 - (a+1)x_4 - (b-1)x_5 = -5\\
0x_1 + 0x_2 - 10x_3 - 4x_4 - 4x_5 = -2\\
6x_1 - 9x_2 + 34x_3 + 0x_4 + 1x_5 = -21}$

Recuerde que $a$ es el último dígito de su código y que $b$ es el penúltimo dígito de su código.

Coloque el  procedimiento como se hizo en este cuaderno.


Se calificará:
* La rutina `solucion_general(parametros)`.
* La tupla de tuplas `soluciones_particulares`